### 1. 날짜 필터링

소속사별 그룹핑하여 스캔들 날짜 범위 확인

In [1]:
import pandas as pd

In [29]:
df = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_크롤링.csv", parse_dates=['사건 날짜'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453 entries, 0 to 452
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   소속사     453 non-null    object        
 1   그룹      307 non-null    object        
 2   연예인 이름  453 non-null    object        
 3   사건 날짜   453 non-null    datetime64[ns]
 4   사건 내용   453 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 17.8+ KB


In [30]:
group = df.groupby('소속사')['사건 날짜'].agg(['min', 'max']).reset_index()
group

,소속사,min,max
0,HYBE,2016-07-06,2024-09-07
1,JYP,2012-07-24,2025-03-02
2,RBW,2018-11-27,2023-10-31
3,SM,1984-01-08,2025-04-16
4,SM C&C,1970-03-09,2023-08-15
5,YG,2007-07-29,2025-01-16
6,아티스트컴퍼니,1999-09-13,2024-11-24
7,큐브,2010-03-31,2023-12-15
8,키이스트,1991-11-19,2025-12-31


In [31]:
hybe = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/주가/HYBE.csv", parse_dates=['날짜'])
hybe_min = hybe['날짜'].agg(['min', 'max'])

In [32]:
len(group)

9

In [33]:
def date_filter(df, stock_dir):

    result_list = []
    grouped = df.groupby('소속사')

    for name, group in grouped:
        
        stock_path = f"{stock_dir}/{name}.csv"
        stock = pd.read_csv(stock_path, parse_dates=['날짜'])
        stock_min = stock['날짜'].min()
        stock_max = stock['날짜'].max()

        filtered = group[(group['사건 날짜']>= stock_min)
                        &(group['사건 날짜']<= stock_max)]
        result_list.append(filtered)
        
    result = pd.concat(result_list, ignore_index=True)
    return result

In [34]:
stock_dir = "C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/주가"
filtered_df = date_filter(df, stock_dir)
filtered_df.to_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/ALL논란_Filtered.csv", index=False, encoding="utf-8-sig")
print(filtered_df.head())

    소속사     그룹 연예인 이름      사건 날짜  \
0  HYBE  방탄소년단     지민 2022-04-25   
1  HYBE  방탄소년단     정국 2024-01-06   
2  HYBE   엔하이픈     정원 2021-11-18   
3  HYBE   엔하이픈     정원 2021-11-19   
4  HYBE   엔하이픈     제이 2023-01-10   

                                               사건 내용  
0  2022년 4월 25일, 언론사 비즈한국의 취재결과 건강보험료 체납으로 보유하고 있...  
1  2024년 1월 6일, 정국이 입대 후 신병 교육을 받는 동안 발생했다. 탈취범은 ...  
2  2021년 11월 18일 오후 5시 , ENHYPEN 은 D-1 이라는 제목으로 V...  
3  2021년 11월 19일 사과문을 게시했다. [ 펼치기 · 접기 ] 엔진 여러분 안...  
4  2023년 1월 10일, 멤버 성훈과 함께 진행한 위버스 라이브 도중 요즘 한국사 ...  


필터링 이전

In [35]:
len(df)

453

필터링 이후

In [36]:
len(filtered_df)

390

### 2. 중복행 필터링

같은 내용이 같은 그룹 멤버에게 들어간 경우 확인

In [37]:
dupes = filtered_df[filtered_df['사건 내용'].duplicated(keep=False)]
grouped_dupes = dupes.groupby('사건 내용')

for content, group in grouped_dupes:
    print("중복된 사건 내용:", content)
    print(group)
    print("-" * 80)

중복된 사건 내용: 2014년 10월 14일 뉴스엔 과의 인터뷰에서 의외의 이야기를 했다. "나는 관심을 무서워 하는 스타일이라 연기를 생각하지도 않았고 연예인이 되고 싶지도 않았다. 연예인이 되어버리면 정말 '왕관을 쓰려는 자, 그 무게를 견뎌라'라는 말 그대로 되니까 무서웠다. 실제 나는 지혜롭지 못하고 덤벙거린다" 또 이성경은 "사람들 앞에 서기에는 많이 부족하다"라며 "그래서 왕관을 안 쓰고 무게 없이 편하게 살겠다는 주의였다" 고 한다.
    소속사   그룹 연예인 이름      사건 날짜  \
245  YG  NaN    이성경 2014-10-14   
264  YG  NaN    이성경 2014-10-14   

                                                 사건 내용  
245  2014년 10월 14일 뉴스엔 과의 인터뷰에서 의외의 이야기를 했다. "나는 관심...  
264  2014년 10월 14일 뉴스엔 과의 인터뷰에서 의외의 이야기를 했다. "나는 관심...  
--------------------------------------------------------------------------------
중복된 사건 내용: 2014년 12월 17일 케이블채널 On Style '겟잇뷰티2014'에 출연해 인터뷰를 진행했다. 당시 이성경은 "둥근 얼굴과 낮은 코, 크지 않은 눈, 넓은 미간까지 단점이 많다. 메이크업으로 가리고 그림을 그려도 안 예쁘다"고 자신의 외모를 평가했다. 이어 그는 "사진에서도 예쁘게 잘 나온다"는 말에 "셀카 잘 나오는 비결은 뽀샤시 어플과 얼짱 각도, 화장발 덕분"이라고 겸손한 태도를 보여 눈길을 끌었다. 이 소식을 접한 네티즌은 "이성경, 무슨 망언이냐", "이성경, 신비로운 이미지인듯", "이성경, 태어났을때부터 모델이었을 거 같아"라는 반응을 보였다.
    소속사   그룹 연예인 이름      사건 날짜  \
246  YG  NaN    이성경 2014-12-17

In [41]:
# 수동으로 삭제 후 확인
filtered_df = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Filtered.csv")
dupes = filtered_df[filtered_df['사건 내용'].duplicated(keep=False)]
grouped_dupes = dupes.groupby('사건 내용')

for content, group in grouped_dupes:
    print("중복된 사건 내용:", content)
    print(group)
    print("-" * 80)

중복된 사건 내용: 2014년 10월 14일 뉴스엔 과의 인터뷰에서 의외의 이야기를 했다. "나는 관심을 무서워 하는 스타일이라 연기를 생각하지도 않았고 연예인이 되고 싶지도 않았다. 연예인이 되어버리면 정말 '왕관을 쓰려는 자, 그 무게를 견뎌라'라는 말 그대로 되니까 무서웠다. 실제 나는 지혜롭지 못하고 덤벙거린다" 또 이성경은 "사람들 앞에 서기에는 많이 부족하다"라며 "그래서 왕관을 안 쓰고 무게 없이 편하게 살겠다는 주의였다" 고 한다.
    소속사   그룹 연예인 이름       사건 날짜  \
244  YG  NaN    이성경  2014-10-14   
263  YG  NaN    이성경  2014-10-14   

                                                 사건 내용  
244  2014년 10월 14일 뉴스엔 과의 인터뷰에서 의외의 이야기를 했다. "나는 관심...  
263  2014년 10월 14일 뉴스엔 과의 인터뷰에서 의외의 이야기를 했다. "나는 관심...  
--------------------------------------------------------------------------------
중복된 사건 내용: 2014년 12월 17일 케이블채널 On Style '겟잇뷰티2014'에 출연해 인터뷰를 진행했다. 당시 이성경은 "둥근 얼굴과 낮은 코, 크지 않은 눈, 넓은 미간까지 단점이 많다. 메이크업으로 가리고 그림을 그려도 안 예쁘다"고 자신의 외모를 평가했다. 이어 그는 "사진에서도 예쁘게 잘 나온다"는 말에 "셀카 잘 나오는 비결은 뽀샤시 어플과 얼짱 각도, 화장발 덕분"이라고 겸손한 태도를 보여 눈길을 끌었다. 이 소식을 접한 네티즌은 "이성경, 무슨 망언이냐", "이성경, 신비로운 이미지인듯", "이성경, 태어났을때부터 모델이었을 거 같아"라는 반응을 보였다.
    소속사   그룹 연예인 이름       사건 날짜  \
245  YG  NaN    이성경  2014-

나머지 중복행 삭제

In [42]:
df = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Filtered.csv")
df_cleaned = df.drop_duplicates(subset=['사건 내용'], keep='first')
df_cleaned.to_csv("ALL논란_Final.csv", index=False, encoding='utf-8-sig')

In [43]:
len(df_cleaned)

363